## Downloading ArmanEmo Dataset

In [ ]:
! git clone https://github.com/Arman-Rayan-Sharif/arman-text-emotion.git
! cp /content/arman-text-emotion/dataset/test.tsv /content/test.tsv
! cp /content/arman-text-emotion/dataset/train.tsv /content/train.tsv
!rm -rf /content/arman-text-emotion

Cloning into 'arman-text-emotion'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 52 (delta 10), reused 9 (delta 2), pack-reused 0
Unpacking objects: 100% (52/52), 572.37 KiB | 5.90 MiB/s, done.


## Installing Requirements

In [ ]:
! pip install openai
! pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Creating Datasets

In [ ]:
import pandas as pd

train = pd.read_csv("/content/train.tsv", header=None, sep="\t")
test = pd.read_csv("/content/test.tsv", header=None, sep="\t")

train.columns = ['text', 'label']
test.columns = ['text', 'label']

print(train.head())

print("---------------------------------------------------------------------")

print(test.head())

                                                text  label
0  خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...    SAD
1     از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم   HATE
2   "کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم...    SAD
3  چون همش با دوربین ثبت شده ، ایا میشه اعتراض زد...  OTHER
4              اين وضع ب طرز خنده داري گريه داره ...    SAD
---------------------------------------------------------------------
                                                text  label
0  اين شايد اولين عزاى عمومى واقعى است كه ياد دار...    SAD
1  دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...  HAPPY
2   کدوم شعبه پول نداده، بگو الان برات آمار دقیق ...  OTHER
3  امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...  HAPPY
4  امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...    SAD


In [ ]:
print(train['label'].value_counts())
print("---------------------------------------------------------------------")
print(test['label'].value_counts())

OTHER       1681
ANGRY        923
SAD          896
FEAR         757
SURPRISE     739
HAPPY        618
HATE         511
Name: label, dtype: int64
---------------------------------------------------------------------
HAPPY       275
SAD         262
OTHER       193
ANGRY       154
SURPRISE    145
HATE         65
FEAR         57
Name: label, dtype: int64


In [ ]:
n_test_sample = 1
test_prompt = train.groupby('label').apply(lambda x: x.sample(n_test_sample))

test_prompt.head()

,,text,label
label,,,
ANGRY,3969,نه توییت جدیدم رو نزدی منشن هم ندادی,ANGRY
FEAR,2884,یکی از چیزایی که روم تاثیر میزاره اینجا اینه ک...,FEAR
HAPPY,3677,مرسیی,HAPPY
HATE,4415,گندشو درورده و منزجرکننده شده,HATE
OTHER,5016,استاد رائفی پور,OTHER


In [ ]:
n_sample_each_group = 3
demonstration = train.groupby('label').apply(lambda x: x.sample(n_sample_each_group))
demonstration = demonstration.reset_index(drop=True)
demonstration = demonstration.sample(frac=1).reset_index(drop=True)

demonstration.head()

,text,label
0,از اینهایی که تو بیوشون مینویسن بک ندی حتماااا...,FEAR
1,سرگردان هستيم .در حاليكه يك ذره آگاهى ، همه را...,SAD
2,چه خوب ميشه اگر اين وسواس رو بخرج بديم و يك با...,OTHER
3,اصلا نمیفهمم چی میگه حتی یه دقیقهای هم نمیتونم...,HATE
4,این ماجراها برای شما اعصاب خوردکنی هست برای ما...,HAPPY


## OpenAI Prompt Methodology

In [ ]:
import openai
openai.api_key  = ""

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message["content"]

## Prompts

In [ ]:
import time

max_retries = 5

### Zero-shot Sample Test

In [ ]:
prompt = f"""
The below sentence is a person's tweet. The tweet is in Persian. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")


Tweet: چه کت وشلوار های قهوه‌ای که ملت تنت میکنن    دمشون گرم ...    این بوی تهران برا همین کت وشلوارهای قهواییه  که ملت تن شما افساد طلبها میکنن هست ...
Model Response: OTHER
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: چقدر ترسناک  
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: نميدونم از شدت خنده چه گهي بخورم
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: شاید ویژگی بدی باشه اما از گریه کردن جلوی کسی متنفرم 
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: بعدها مشخص شد که یه دستور داشت تو تنظیمات مودم باید تایپ میکردی که صداشو قطع میکرد  نمیدونم چی‌چی رو مساوی صفر میکردی از صدای قیژویژوبلنذبلتتد خلاص میشدی  
Model Response: FRUSTRATION
Actual Label: OTHER
---------------------------------------------------------------------

Correct Answers: 4/7

In [ ]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")


Tweet: #جهانگیری انقدرخایه ندارید بگیدتمام این بدبختی ها مشکلات ناشی ازانقلاب شکوهمنداسلامی وسومدیریت درتمامی ارکان این نظام بوده بوده وگرنه با این سیل ازنیروی جوان وتحصیل کرده ایران ده ها سال اززمان الانش جلوتربود
Model Response: ANGRY
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: فوبیا کیک شکلاتی 
Model Response: OTHER
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: خوب بود به نظرم رفتم تجربه خوبی شد  
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet:  بدم ازت میاد
Model Response: SAD
Actual Label: HATE
---------------------------------------------------------------------
Tweet: البته طبق حرف قدیمت، حرف من همون حرف خودته 
Model Response: OTHER
Actual Label: OTHER
---------------------------------------------------------------------
Tweet: یارو بیخیال من شو من هرشب بیدارم بابا من خوابم میاد 
Model Response: SAD
Actual L

Correct Answers: 4/7

In [1]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")


NameError: ignored

Correct Answers: 4/7

In [ ]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: دیگر، عصبانی، غمگین، ترس، غافلگیری، خوشحالی، نفرت. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از دیگر، استفاده کنید. \

توییت: SAMPLE_TWEET
احساس:
"""

emotion_persian_to_english = {
    "دیگر": "OTHER",
    "عصبانی": "ANGRY",
    "غمگین": "SAD",
    "ترس": "FEAR",
    "غافلگیری": "SURPRISE",
    "خوشحالی": "HAPPY",
    "نفرت": "HATE"
}

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            if response not in emotion_persian_to_english.keys():
              print(f"Model Response: NotFound {response}")
            else:
              print(f"Model Response: {emotion_persian_to_english[response]}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")


Tweet: #جهانگیری انقدرخایه ندارید بگیدتمام این بدبختی ها مشکلات ناشی ازانقلاب شکوهمنداسلامی وسومدیریت درتمامی ارکان این نظام بوده بوده وگرنه با این سیل ازنیروی جوان وتحصیل کرده ایران ده ها سال اززمان الانش جلوتربود
Model Response: HATE
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: فوبیا کیک شکلاتی 
Model Response: SURPRISE
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: خوب بود به نظرم رفتم تجربه خوبی شد  
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet:  بدم ازت میاد
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: البته طبق حرف قدیمت، حرف من همون حرف خودته 
Model Response: SURPRISE
Actual Label: OTHER
---------------------------------------------------------------------
Tweet: یارو بیخیال من شو من هرشب بیدارم بابا من خوابم میاد 
Model Response: SURPRI

Correct Answers: 2/7

In [ ]:
from googletrans import Translator

prompt = f"""
The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            translator = Translator()
            result = translator.translate(row['text'], src='fa', dest='en')

            response = get_completion(prompt.replace("SAMPLE_TWEET", result.text))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")


Tweet: #جهانگیری انقدرخایه ندارید بگیدتمام این بدبختی ها مشکلات ناشی ازانقلاب شکوهمنداسلامی وسومدیریت درتمامی ارکان این نظام بوده بوده وگرنه با این سیل ازنیروی جوان وتحصیل کرده ایران ده ها سال اززمان الانش جلوتربود
Model Response: ANGRY
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: فوبیا کیک شکلاتی 
Model Response: OTHER (The sentence does not include any specific emotion, it is just a statement about the person's fear of chocolate cake.)
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: خوب بود به نظرم رفتم تجربه خوبی شد  
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet:  بدم ازت میاد
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: البته طبق حرف قدیمت، حرف من همون حرف خودته 
Model Response: OTHER
Actual Label: OTHER
-------------------------------------

Correct Answers: 4/7

### Zero-shot Metric

In [ ]:
acceptable_classes = [
    "OTHER",
    "ANGRY",
    "SAD",
    "FEAR",
    "SURPRISE",
    "HAPPY",
    "HATE"
]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=1)
    precision = precision_score(actual, predicted, average='weighted', zero_division=1)
    recall = recall_score(actual, predicted, average='weighted', zero_division=1)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = set(actual + predicted)

    metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall,
        'Class-wise Counts': {}
    }

    for class_label in classes:
        class_precision = precision_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)
        class_recall = recall_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)
        class_f1 = f1_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)

        metrics['Class-wise Counts'][class_label] = {
            'Actual Count': actual_counts[class_label],
            'Predicted Count': predicted_counts[class_label],
            'Precision': class_precision,
            'Recall': class_recall,
            'F1 Score': class_f1
        }

    for metric, value in metrics.items():
        if metric == 'Class-wise Counts':
            print(f"\n{metric}:")
            for class_label, stats in value.items():
                print(f"Class: {class_label}")
                for stat, val in stats.items():
                    print(f"{stat}: {val}")
                print()
        else:
            print(f"{metric}: {value}")

    return metrics

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
The below sentence is a person's tweet. The tweet is in Persian. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

 13%|█▎        | 147/1151 [01:55<08:27,  1.98it/s]

An error occurred: The server had an error while processing your request. Sorry about that!
Retrying (1/5)...


 16%|█▌        | 181/1151 [02:47<24:27,  1.51s/it]

An error occurred: The server had an error while processing your request. Sorry about that!
Retrying (1/5)...


 27%|██▋       | 314/1151 [04:50<07:08,  1.95it/s]

...

 89%|████████▉ | 1028/1151 [23:21<03:37,  1.76s/it]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 67fac2d2fba580ea17e96c1d719c85e1 in your message.)
Retrying (1/5)...


 91%|█████████ | 1044/1151 [24:28<00:59,  1.81it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1dbd46169ba6a851176836ce58389f31 in your message.)
Retrying (1/5)...


100%|██████████| 1151/1151 [27:27<00:00,  1.43s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.5994787141615986
F1 Score: 0.5674127351052752
Precision: 0.6417517088132937
Recall: 0.5994787141615986

Class-wise Counts:
Class: OTHER
Actual Count: 193
Predicted Count: 150
Precision: 0.44
Recall: 0.34196891191709844
F1 Score: 0.3848396501457726

Class: HATE
Actual Count: 65
Predicted Count: 34
Precision: 0.6176470588235294
Recall: 0.3230769230769231
F1 Score: 0.42424242424242425

Class: ANGRY
Actual Count: 154
Predicted Count: 273
Precision: 0.43956043956043955
Recall: 0.7792207792207793
F1 Score: 0.5620608899297423

Class: FEAR
Actual Count: 57
Predicted Count: 45
Precision: 0.8
Recall: 0.631578947368421
F1 Score: 0.7058823529411765

Class: HAPPY
Actual Count: 275
Predicted Count: 252
Precision: 0.8412698412698413
Recall: 0.7709090909090909
F1 Score: 0.8045540796963947

Class: SAD
Actual Count: 262
Predicted Count: 382
Precision: 0.5837696335078534
Recall: 0.851145038167939
F1 Score: 0.6925465838509317

Class: SURPRISE
Actual Count: 145
Predicted Count: 15
Precision: 0.

{'Accuracy': 0.5994787141615986,
 'F1 Score': 0.5674127351052752,
 'Precision': 0.6417517088132937,
 'Recall': 0.5994787141615986,
 'Class-wise Counts': {'OTHER': {'Actual Count': 193,
   'Predicted Count': 150,
   'Precision': 0.44,
   'Recall': 0.34196891191709844,
   'F1 Score': 0.3848396501457726},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 34,
   'Precision': 0.6176470588235294,
   'Recall': 0.3230769230769231,
   'F1 Score': 0.42424242424242425},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 273,
   'Precision': 0.43956043956043955,
   'Recall': 0.7792207792207793,
   'F1 Score': 0.5620608899297423},
  'FEAR': {'Actual Count': 57,
   'Predicted Count': 45,
   'Precision': 0.8,
   'Recall': 0.631578947368421,
   'F1 Score': 0.7058823529411765},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 252,
   'Precision': 0.8412698412698413,
   'Recall': 0.7709090909090909,
   'F1 Score': 0.8045540796963947},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 382

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

  1%|▏         | 17/1151 [00:12<13:17,  1.42it/s]

An error occurred: The server had an error while processing your request. Sorry about that!
Retrying (1/5)...


  4%|▍         | 45/1151 [00:56<09:02,  2.04it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 48e07abbbd0d207c8252f8eb3137823e in your message.)
Retrying (1/5)...


  6%|▋         | 74/1151 [02:11<09:38,  1.86it/s]

...

 84%|████████▍ | 966/1151 [20:38<01:36,  1.91it/s]

An error occurred: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Sat, 17 Jun 2023 10:00:14 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7d8a6a2e7abaa9b5-TPE', 'alt-svc': 'h3=":443"; ma=86400'}
Retrying (1/5)...


 94%|█████████▎| 1077/1151 [27:23<00:39,  1.88it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID de2a8c3dcd7cb8fb0e76c3626a1de415 in your message.)
Retrying (1/5)...


100%|██████████| 1151/1151 [28:59<00:00,  1.51s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.5664639443961772
F1 Score: 0.5576689089618787
Precision: 0.6146044011565439
Recall: 0.5664639443961772

Class-wise Counts:
Class: OTHER
Actual Count: 193
Predicted Count: 279
Precision: 0.2939068100358423
Recall: 0.42487046632124353
F1 Score: 0.34745762711864403

Class: HATE
Actual Count: 65
Predicted Count: 43
Precision: 0.5116279069767442
Recall: 0.3384615384615385
F1 Score: 0.40740740740740744

Class: ANGRY
Actual Count: 154
Predicted Count: 214
Precision: 0.4672897196261682
Recall: 0.6493506493506493
F1 Score: 0.5434782608695652

Class: FEAR
Actual Count: 57
Predicted Count: 35
Precision: 0.6857142857142857
Recall: 0.42105263157894735
F1 Score: 0.5217391304347826

Class: HAPPY
Actual Count: 275
Predicted Count: 283
Precision: 0.7879858657243817
Recall: 0.8109090909090909
F1 Score: 0.7992831541218639

Class: SAD
Actual Count: 262
Predicted Count: 267
Precision: 0.6629213483146067
Recall: 0.6755725190839694
F1 Score: 0.6691871455576558

Class: SURPRISE
Actual Count: 145
P

{'Accuracy': 0.5664639443961772,
 'F1 Score': 0.5576689089618787,
 'Precision': 0.6146044011565439,
 'Recall': 0.5664639443961772,
 'Class-wise Counts': {'OTHER': {'Actual Count': 193,
   'Predicted Count': 279,
   'Precision': 0.2939068100358423,
   'Recall': 0.42487046632124353,
   'F1 Score': 0.34745762711864403},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 43,
   'Precision': 0.5116279069767442,
   'Recall': 0.3384615384615385,
   'F1 Score': 0.40740740740740744},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 214,
   'Precision': 0.4672897196261682,
   'Recall': 0.6493506493506493,
   'F1 Score': 0.5434782608695652},
  'FEAR': {'Actual Count': 57,
   'Predicted Count': 35,
   'Precision': 0.6857142857142857,
   'Recall': 0.42105263157894735,
   'F1 Score': 0.5217391304347826},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 283,
   'Precision': 0.7879858657243817,
   'Recall': 0.8109090909090909,
   'F1 Score': 0.7992831541218639},
  'SAD': {'Actual Count':

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

 13%|█▎        | 151/1151 [01:28<10:52,  1.53it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 912aa05e74c5343e94b45ad2883abba2 in your message.)
Retrying (1/5)...


 15%|█▌        | 173/1151 [02:42<07:31,  2.17it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 0e90f4f18f81673f2dd3f25c96bd2532 in your message.)
Retrying (1/5)...


 32%|███▏      | 365/1151 [05:27<06:37,  1.98it/s]

...

 85%|████████▌ | 983/1151 [24:16<01:18,  2.13it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 84505b27a8ce1920f1a0f7fed7f7f81f in your message.)
Retrying (1/5)...


 95%|█████████▍| 1091/1151 [26:26<00:28,  2.09it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 600ce7779e3093152687007fa1673986 in your message.)
Retrying (1/5)...


100%|██████████| 1151/1151 [28:01<00:00,  1.46s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.5421372719374457
F1 Score: 0.5293810816669298
Precision: 0.5980866144848542
Recall: 0.5421372719374457

Class-wise Counts:
Class: OTHER
Actual Count: 193
Predicted Count: 307
Precision: 0.2703583061889251
Recall: 0.43005181347150256
F1 Score: 0.332

Class: HATE
Actual Count: 65
Predicted Count: 40
Precision: 0.5
Recall: 0.3076923076923077
F1 Score: 0.380952380952381

Class: ANGRY
Actual Count: 154
Predicted Count: 198
Precision: 0.4090909090909091
Recall: 0.525974025974026
F1 Score: 0.46022727272727276

Class: FEAR
Actual Count: 57
Predicted Count: 36
Precision: 0.6666666666666666
Recall: 0.42105263157894735
F1 Score: 0.5161290322580646

Class: HAPPY
Actual Count: 275
Predicted Count: 308
Precision: 0.7727272727272727
Recall: 0.8654545454545455
F1 Score: 0.8164665523156088

Class: SAD
Actual Count: 262
Predicted Count: 243
Precision: 0.6707818930041153
Recall: 0.6221374045801527
F1 Score: 0.6455445544554457

Class: SURPRISE
Actual Count: 145
Predicted Count: 19
Precision: 0

{'Accuracy': 0.5421372719374457,
 'F1 Score': 0.5293810816669298,
 'Precision': 0.5980866144848542,
 'Recall': 0.5421372719374457,
 'Class-wise Counts': {'OTHER': {'Actual Count': 193,
   'Predicted Count': 307,
   'Precision': 0.2703583061889251,
   'Recall': 0.43005181347150256,
   'F1 Score': 0.332},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 40,
   'Precision': 0.5,
   'Recall': 0.3076923076923077,
   'F1 Score': 0.380952380952381},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 198,
   'Precision': 0.4090909090909091,
   'Recall': 0.525974025974026,
   'F1 Score': 0.46022727272727276},
  'FEAR': {'Actual Count': 57,
   'Predicted Count': 36,
   'Precision': 0.6666666666666666,
   'Recall': 0.42105263157894735,
   'F1 Score': 0.5161290322580646},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 308,
   'Precision': 0.7727272727272727,
   'Recall': 0.8654545454545455,
   'F1 Score': 0.8164665523156088},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 243

In [ ]:
from tqdm import tqdm
from googletrans import Translator

actual = []
predicted = []

prompt = f"""
The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            translator = Translator()
            result = translator.translate(row['text'], src='fa', dest='en')

            response = get_completion(prompt.replace("SAMPLE_TWEET", result.text))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            if "JSON" not in str(e):
              print(f"An error occurred: {str(e)}")
              print(f"Retrying ({retries + 1}/{max_retries})...")

            retries += 1
            time.sleep(10)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

  7%|▋         | 80/1151 [00:48<07:36,  2.35it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e61bf0a4a856946d188932c964911906 in your message.)
Retrying (1/5)...


 11%|█▏        | 132/1151 [02:02<16:50,  1.01it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7261d9671a63d90a1fbbb8ea1b231164 in your message.)
Retrying (1/5)...


 20%|██        | 233/1151 [03:32<07:59,  1.91it/s]

...

 56%|█████▌    | 643/1151 [08:43<06:04,  1.39it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 0d2954bddedbb7873265b53e6bf9b62c in your message.)
Retrying (1/5)...


 60%|█████▉    | 690/1151 [09:48<03:31,  2.18it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 16759b3ea2c1e4518040d850597eabbb in your message.)
Retrying (1/5)...


100%|██████████| 1151/1151 [22:28<00:00,  1.17s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.4943527367506516
F1 Score: 0.4948624095145838
Precision: 0.5784410120962653
Recall: 0.4943527367506516

Class-wise Counts:
Class: SAD
Actual Count: 262
Predicted Count: 222
Precision: 0.6801801801801802
Recall: 0.5763358778625954
F1 Score: 0.6239669421487604

Class: ANGRY
Actual Count: 154
Predicted Count: 229
Precision: 0.35807860262008734
Recall: 0.5324675324675324
F1 Score: 0.4281984334203655

Class: SURPRISE
Actual Count: 145
Predicted Count: 19
Precision: 0.5789473684210527
Recall: 0.07586206896551724
F1 Score: 0.13414634146341461

Class: HAPPY
Actual Count: 275
Predicted Count: 201
Precision: 0.7910447761194029
Recall: 0.5781818181818181
F1 Score: 0.6680672268907563

Class: HATE
Actual Count: 65
Predicted Count: 43
Precision: 0.5581395348837209
Recall: 0.36923076923076925
F1 Score: 0.4444444444444445

Class: OTHER
Actual Count: 193
Predicted Count: 393
Precision: 0.2824427480916031
Recall: 0.5751295336787565
F1 Score: 0.3788395904436861

Class: FEAR
Actual Count: 57
P

{'Accuracy': 0.4943527367506516,
 'F1 Score': 0.4948624095145838,
 'Precision': 0.5784410120962653,
 'Recall': 0.4943527367506516,
 'Class-wise Counts': {'SAD': {'Actual Count': 262,
   'Predicted Count': 222,
   'Precision': 0.6801801801801802,
   'Recall': 0.5763358778625954,
   'F1 Score': 0.6239669421487604},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 229,
   'Precision': 0.35807860262008734,
   'Recall': 0.5324675324675324,
   'F1 Score': 0.4281984334203655},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 19,
   'Precision': 0.5789473684210527,
   'Recall': 0.07586206896551724,
   'F1 Score': 0.13414634146341461},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 201,
   'Precision': 0.7910447761194029,
   'Recall': 0.5781818181818181,
   'F1 Score': 0.6680672268907563},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 43,
   'Precision': 0.5581395348837209,
   'Recall': 0.36923076923076925,
   'F1 Score': 0.4444444444444445},
  'OTHER': {'Actual Co

### Few-shot Sample Test

In [ ]:
def create_demonstration_prompt(demonstration_df, title="Example",
                                input="Tweet", label="Emotion",
                                label_map=None):
  result = ""
  i = 1
  for index, row in demonstration_df.iterrows():
    result += f"{title} {i}:\n"
    result += f"{input}: {row['text']}\n"
    if label_map is None:
      result += f"{label}: {row['label']}\n"
    else:
      result += f"{label}: {label_map[row['label']]}\n"
    i += 1
  return result

In [ ]:
print(create_demonstration_prompt(demonstration))

Example 1:
Tweet: از اینهایی که تو بیوشون مینویسن بک ندی حتمااااااا آنفالویی خیلی میترسم   و از ترسم حتمااااااا بک نمیدم 
Emotion: FEAR
Example 2:
Tweet: سرگردان هستيم .در حاليكه يك ذره آگاهى ، همه را از سرگردانى نجات مى دهد.. . #طاهرى_خزعلى
Emotion: SAD
Example 3:
Tweet: چه خوب ميشه اگر اين وسواس رو بخرج بديم و يك بار براى هميشه واژه " مرگ" و هر لغتى كه در پس خودش بار معنايى حذف رو داره از شعار هامون كنار بگذاريم، و بازتوليد خشونت كلامى رو پايان بديم. #چالش_دعوت_به_تظاهرات_سراسری
Emotion: OTHER
Example 4:
Tweet: اصلا نمیفهمم چی میگه حتی یه دقیقهای هم نمیتونم تحملش کنم، بعدش برام آرزو موفقیت میکنه
Emotion: HATE
Example 5:
Tweet: این ماجراها برای شما اعصاب خوردکنی هست برای ما سوژه خنده    4صبح آمده سلام کنه  
Emotion: HAPPY
Example 6:
Tweet: بعد جمهوری اسهالیم وضع لنگو می‌بینیم  
Emotion: SAD
Example 7:
Tweet: چرا آخه  
Emotion: SURPRISE
Example 8:
Tweet: یه پسره هست هرروز میاد ماشینش میذاره پارکینگ انقدرررررر ازش بدم میاد چجوری بکشمش 
Emotion: HATE
Example 9:
Tweet: اون دو نفری که منتظ

In [ ]:
prompt = f"""
The below sentence is a person's tweet. The tweet is in Persian. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: نه توییت جدیدم رو نزدی    منشن هم ندادی  
Model Response: ANGRY
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: یکی از چیزایی که روم تاثیر میزاره اینجا اینه که بعضی ها که اینجا از افسرگیشون یا حالتشون میگم اگاهانه یا ناخوداگاه سعی میکنم همون حس و بگیرم و اون حس درم بوجود میاد و با اینکه ارزوی مرگ و داشتم میترسم وقتی یکی اینجا میخواد خودکشی کنه نمیدونم بخاطر اون یا چون من جراتشو نداشتم
Model Response: SAD
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: مرسیی 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: گندشو درورده و منزجرکننده شده  
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: استاد رائفی پور  
Model Response: OTHER
Actual Label: OTHER
---------------------------------------------------------------------
Tweet:  بهتر نبود بجا تهدید مینوشتی

Correct Answers: 5/7

In [ ]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: نه توییت جدیدم رو نزدی    منشن هم ندادی  
Model Response: SAD
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: یکی از چیزایی که روم تاثیر میزاره اینجا اینه که بعضی ها که اینجا از افسرگیشون یا حالتشون میگم اگاهانه یا ناخوداگاه سعی میکنم همون حس و بگیرم و اون حس درم بوجود میاد و با اینکه ارزوی مرگ و داشتم میترسم وقتی یکی اینجا میخواد خودکشی کنه نمیدونم بخاطر اون یا چون من جراتشو نداشتم
Model Response: SAD
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: مرسیی 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: گندشو درورده و منزجرکننده شده  
Model Response: SAD
Actual Label: HATE
---------------------------------------------------------------------
Tweet: استاد رائفی پور  
Model Response: OTHER
Actual Label: OTHER
---------------------------------------------------------------------
Tweet:  بهتر نبود بجا تهدید مینوشتی چه

Correct Answers: 3/7

In [ ]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: نه توییت جدیدم رو نزدی    منشن هم ندادی  
Model Response: SAD
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: یکی از چیزایی که روم تاثیر میزاره اینجا اینه که بعضی ها که اینجا از افسرگیشون یا حالتشون میگم اگاهانه یا ناخوداگاه سعی میکنم همون حس و بگیرم و اون حس درم بوجود میاد و با اینکه ارزوی مرگ و داشتم میترسم وقتی یکی اینجا میخواد خودکشی کنه نمیدونم بخاطر اون یا چون من جراتشو نداشتم
Model Response: SAD
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: مرسیی 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: گندشو درورده و منزجرکننده شده  
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: استاد رائفی پور  
Model Response: OTHER
Actual Label: OTHER
---------------------------------------------------------------------
Tweet:  بهتر نبود بجا تهدید مینوشتی چ

Correct Answer: 4/7

In [ ]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration, title='مثال', input="توییت", label="احساس")}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: نه توییت جدیدم رو نزدی    منشن هم ندادی  
Model Response: SAD
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: یکی از چیزایی که روم تاثیر میزاره اینجا اینه که بعضی ها که اینجا از افسرگیشون یا حالتشون میگم اگاهانه یا ناخوداگاه سعی میکنم همون حس و بگیرم و اون حس درم بوجود میاد و با اینکه ارزوی مرگ و داشتم میترسم وقتی یکی اینجا میخواد خودکشی کنه نمیدونم بخاطر اون یا چون من جراتشو نداشتم
Model Response: SAD
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: مرسیی 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: گندشو درورده و منزجرکننده شده  
Model Response: SAD
Actual Label: HATE
---------------------------------------------------------------------
Tweet: استاد رائفی پور  
Model Response: OTHER
Actual Label: OTHER
---------------------------------------------------------------------
Tweet:  بهتر نبود بجا تهدید مینوشتی چه

Correct Answer: 3/7

In [ ]:
emotion_persian_to_english = {
    "دیگر": "OTHER",
    "عصبانی": "ANGRY",
    "غمگین": "SAD",
    "ترس": "FEAR",
    "غافلگیری": "SURPRISE",
    "خوشحالی": "HAPPY",
    "نفرت": "HATE"
}

emotion_english_to_persian = {
    "OTHER": "دیگر",
    "ANGRY": "عصبانی",
    "SAD": "غمگین",
    "FEAR": "ترس",
    "SURPRISE": "غافلگیری",
    "HAPPY": "خوشحالی",
    "HATE": "نفرت"
}

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration, label_map=emotion_english_to_persian)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            if response not in emotion_persian_to_english.keys():
              print(f"Model Response: NotFound {response}")
            else:
              print(f"Model Response: {emotion_persian_to_english[response]}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: نه توییت جدیدم رو نزدی    منشن هم ندادی  
Model Response: NotFound غمگینی
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: یکی از چیزایی که روم تاثیر میزاره اینجا اینه که بعضی ها که اینجا از افسرگیشون یا حالتشون میگم اگاهانه یا ناخوداگاه سعی میکنم همون حس و بگیرم و اون حس درم بوجود میاد و با اینکه ارزوی مرگ و داشتم میترسم وقتی یکی اینجا میخواد خودکشی کنه نمیدونم بخاطر اون یا چون من جراتشو نداشتم
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: مرسیی 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: گندشو درورده و منزجرکننده شده  
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: استاد رائفی پور  
Model Response: HAPPY
Actual Label: OTHER
---------------------------------------------------------------------
Tweet:  بهتر نبود بجا ته

Correct Answer: 3/7

In [ ]:
emotion_persian_to_english = {
    "دیگر": "OTHER",
    "عصبانی": "ANGRY",
    "غمگین": "SAD",
    "ترس": "FEAR",
    "غافلگیری": "SURPRISE",
    "خوشحالی": "HAPPY",
    "نفرت": "HATE"
}

emotion_english_to_persian = {
    "OTHER": "دیگر",
    "ANGRY": "عصبانی",
    "SAD": "غمگین",
    "FEAR": "ترس",
    "SURPRISE": "غافلگیری",
    "HAPPY": "خوشحالی",
    "HATE": "نفرت"
}

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration, title='مثال', input="توییت", label="احساس", label_map=emotion_english_to_persian)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            if response not in emotion_persian_to_english.keys():
              print(f"Model Response: NotFound {response}")
            else:
              print(f"Model Response: {emotion_persian_to_english[response]}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: نه توییت جدیدم رو نزدی    منشن هم ندادی  
Model Response: SAD
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: یکی از چیزایی که روم تاثیر میزاره اینجا اینه که بعضی ها که اینجا از افسرگیشون یا حالتشون میگم اگاهانه یا ناخوداگاه سعی میکنم همون حس و بگیرم و اون حس درم بوجود میاد و با اینکه ارزوی مرگ و داشتم میترسم وقتی یکی اینجا میخواد خودکشی کنه نمیدونم بخاطر اون یا چون من جراتشو نداشتم
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: مرسیی 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: گندشو درورده و منزجرکننده شده  
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: استاد رائفی پور  
Model Response: HAPPY
Actual Label: OTHER
---------------------------------------------------------------------
Tweet:  بهتر نبود بجا تهدید مینوشتی 

Correct Answer: 3/7

In [ ]:
from googletrans import Translator

prompt = f"""
The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            translator = Translator()
            result = translator.translate(row['text'], src='fa', dest='en')

            response = get_completion(prompt.replace("SAMPLE_TWEET", result.text))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: نه توییت جدیدم رو نزدی    منشن هم ندادی  
Model Response: OTHER
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet: یکی از چیزایی که روم تاثیر میزاره اینجا اینه که بعضی ها که اینجا از افسرگیشون یا حالتشون میگم اگاهانه یا ناخوداگاه سعی میکنم همون حس و بگیرم و اون حس درم بوجود میاد و با اینکه ارزوی مرگ و داشتم میترسم وقتی یکی اینجا میخواد خودکشی کنه نمیدونم بخاطر اون یا چون من جراتشو نداشتم
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: مرسیی 
Model Response: OTHER
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: گندشو درورده و منزجرکننده شده  
Model Response: OTHER
Actual Label: HATE
---------------------------------------------------------------------
Tweet: استاد رائفی پور  
Model Response: OTHER
Actual Label: OTHER
---------------------------------------------------------------------
Tweet:  بهتر نبود بجا تهدید مینوش

Correct Answer: 2/7

### Few-shot Metric

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
The below sentence is a person's tweet. The tweet is in Persian. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

  4%|▍         | 50/1151 [00:24<29:11,  1.59s/it]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 08bdd5181683ded0161863b8ccdc1dd4 in your message.)
Retrying (1/5)...


 16%|█▋        | 188/1151 [03:03<05:16,  3.04it/s]

An error occurred: The server had an error while processing your request. Sorry about that!
Retrying (1/5)...


 17%|█▋        | 199/1151 [03:37<08:52,  1.79it/s]

...

 87%|████████▋ | 1000/1151 [33:41<07:04,  2.81s/it]

An error occurred: The server had an error while processing your request. Sorry about that!
Retrying (1/5)...


 94%|█████████▍| 1087/1151 [35:00<00:22,  2.84it/s]

An error occurred: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Sun, 18 Jun 2023 07:54:36 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7d91ef60fb872433-ATL', 'alt-svc': 'h3=":443"; ma=86400'}
Retrying (1/5)...


100%|██████████| 1151/1151 [41:20<00:00,  2.16s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.6220677671589921
F1 Score: 0.5886576484002196
Precision: 0.6678544622696695
Recall: 0.6220677671589921

Class-wise Counts:
Class: SAD
Actual Count: 262
Predicted Count: 334
Precision: 0.6467065868263473
Recall: 0.8244274809160306
F1 Score: 0.7248322147651007

Class: ANGRY
Actual Count: 154
Predicted Count: 264
Precision: 0.45454545454545453
Recall: 0.7792207792207793
F1 Score: 0.5741626794258373

Class: SURPRISE
Actual Count: 145
Predicted Count: 10
Precision: 0.9
Recall: 0.06206896551724138
F1 Score: 0.11612903225806452

Class: HAPPY
Actual Count: 275
Predicted Count: 273
Precision: 0.8461538461538461
Recall: 0.84
F1 Score: 0.8430656934306568

Class: HATE
Actual Count: 65
Predicted Count: 40
Precision: 0.425
Recall: 0.26153846153846155
F1 Score: 0.3238095238095238

Class: OTHER
Actual Count: 193
Predicted Count: 190
Precision: 0.47368421052631576
Recall: 0.46632124352331605
F1 Score: 0.46997389033942555

Class: FEAR
Actual Count: 57
Predicted Count: 40
Precision: 0.825
Rec

{'Accuracy': 0.6220677671589921,
 'F1 Score': 0.5886576484002196,
 'Precision': 0.6678544622696695,
 'Recall': 0.6220677671589921,
 'Class-wise Counts': {'SAD': {'Actual Count': 262,
   'Predicted Count': 334,
   'Precision': 0.6467065868263473,
   'Recall': 0.8244274809160306,
   'F1 Score': 0.7248322147651007},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 264,
   'Precision': 0.45454545454545453,
   'Recall': 0.7792207792207793,
   'F1 Score': 0.5741626794258373},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 10,
   'Precision': 0.9,
   'Recall': 0.06206896551724138,
   'F1 Score': 0.11612903225806452},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 273,
   'Precision': 0.8461538461538461,
   'Recall': 0.84,
   'F1 Score': 0.8430656934306568},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 40,
   'Precision': 0.425,
   'Recall': 0.26153846153846155,
   'F1 Score': 0.3238095238095238},
  'OTHER': {'Actual Count': 193,
   'Predicted Count': 190,
   '

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

 30%|██▉       | 344/1151 [03:25<04:31,  2.97it/s]

An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-I0sUs99dctVXkZZZ2ifht3RO on tokens per min. Limit: 90000 / min. Current: 89260 / min. Contact us through our help center at help.openai.com if you continue to have issues.
Retrying (1/5)...


 47%|████▋     | 537/1151 [05:50<03:47,  2.69it/s]

An error occurred: Rate limit reached for default-gpt-3.5-turbo in organization org-I0sUs99dctVXkZZZ2ifht3RO on tokens per min. Limit: 90000 / min. Current: 89242 / min. Contact us through our help center at help.openai.com if you continue to have issues.
Retrying (1/5)...


 54%|█████▍    | 620/1151 [06:52<03:04,  2.87it/s]

...

 82%|████████▏ | 940/1151 [11:24<02:37,  1.34it/s]

An error occurred: The server had an error while processing your request. Sorry about that!
Retrying (1/5)...


 83%|████████▎ | 959/1151 [12:09<01:11,  2.69it/s]

An error occurred: The server had an error while processing your request. Sorry about that!
Retrying (1/5)...


100%|██████████| 1151/1151 [15:09<00:00,  1.27it/s]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.6220677671589921
F1 Score: 0.5882664113136746
Precision: 0.6719160821356622
Recall: 0.6220677671589921

Class-wise Counts:
Class: SAD
Actual Count: 262
Predicted Count: 325
Precision: 0.6523076923076923
Recall: 0.8091603053435115
F1 Score: 0.7223168654173765

Class: ANGRY
Actual Count: 154
Predicted Count: 315
Precision: 0.4095238095238095
Recall: 0.8376623376623377
F1 Score: 0.5501066098081023

Class: SURPRISE
Actual Count: 145
Predicted Count: 16
Precision: 0.875
Recall: 0.09655172413793103
F1 Score: 0.1739130434782609

Class: HAPPY
Actual Count: 275
Predicted Count: 283
Precision: 0.8303886925795053
Recall: 0.8545454545454545
F1 Score: 0.8422939068100358

Class: HATE
Actual Count: 65
Predicted Count: 19
Precision: 0.5263157894736842
Recall: 0.15384615384615385
F1 Score: 0.2380952380952381

Class: OTHER
Actual Count: 193
Predicted Count: 148
Precision: 0.5472972972972973
Recall: 0.41968911917098445
F1 Score: 0.47507331378299117

Class: FEAR
Actual Count: 57
Predicted Coun

{'Accuracy': 0.6220677671589921,
 'F1 Score': 0.5882664113136746,
 'Precision': 0.6719160821356622,
 'Recall': 0.6220677671589921,
 'Class-wise Counts': {'SAD': {'Actual Count': 262,
   'Predicted Count': 325,
   'Precision': 0.6523076923076923,
   'Recall': 0.8091603053435115,
   'F1 Score': 0.7223168654173765},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 315,
   'Precision': 0.4095238095238095,
   'Recall': 0.8376623376623377,
   'F1 Score': 0.5501066098081023},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 16,
   'Precision': 0.875,
   'Recall': 0.09655172413793103,
   'F1 Score': 0.1739130434782609},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 283,
   'Precision': 0.8303886925795053,
   'Recall': 0.8545454545454545,
   'F1 Score': 0.8422939068100358},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 19,
   'Precision': 0.5263157894736842,
   'Recall': 0.15384615384615385,
   'F1 Score': 0.2380952380952381},
  'OTHER': {'Actual Count': 193,
   '

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Examples:
{create_demonstration_prompt(demonstration)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

  7%|▋         | 83/1151 [00:41<06:12,  2.87it/s]

An error occurred: The server had an error while processing your request. Sorry about that!
Retrying (1/5)...


 10%|▉         | 111/1151 [01:30<06:46,  2.56it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID f17831caaa51f71cfbe318d13b790152 in your message.)
Retrying (1/5)...


 15%|█▌        | 177/1151 [03:19<09:51,  1.65it/s]

...

 96%|█████████▌| 1100/1151 [28:46<00:25,  1.97it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1e347414483fa87b08d9d22f7ec41767 in your message.)
Retrying (1/5)...


 96%|█████████▌| 1105/1151 [29:48<03:36,  4.70s/it]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 254e5552adb2ded95acbbd62d3d53829 in your message.)
Retrying (1/5)...


100%|██████████| 1151/1151 [31:09<00:00,  1.62s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.6151172893136403
F1 Score: 0.5861137115869857
Precision: 0.6375185061396417
Recall: 0.6151172893136403

Class-wise Counts:
Class: SAD
Actual Count: 262
Predicted Count: 311
Precision: 0.6495176848874598
Recall: 0.7709923664122137
F1 Score: 0.7050610820244326

Class: ANGRY
Actual Count: 154
Predicted Count: 266
Precision: 0.4398496240601504
Recall: 0.7597402597402597
F1 Score: 0.557142857142857

Class: SURPRISE
Actual Count: 145
Predicted Count: 25
Precision: 0.76
Recall: 0.1310344827586207
F1 Score: 0.2235294117647059

Class: HAPPY
Actual Count: 275
Predicted Count: 304
Precision: 0.7927631578947368
Recall: 0.8763636363636363
F1 Score: 0.8324697754749568

Class: HATE
Actual Count: 65
Predicted Count: 24
Precision: 0.5
Recall: 0.18461538461538463
F1 Score: 0.2696629213483146

Class: OTHER
Actual Count: 193
Predicted Count: 177
Precision: 0.4519774011299435
Recall: 0.41450777202072536
F1 Score: 0.4324324324324324

Class: FEAR
Actual Count: 57
Predicted Count: 44
Precision: 0.

{'Accuracy': 0.6151172893136403,
 'F1 Score': 0.5861137115869857,
 'Precision': 0.6375185061396417,
 'Recall': 0.6151172893136403,
 'Class-wise Counts': {'SAD': {'Actual Count': 262,
   'Predicted Count': 311,
   'Precision': 0.6495176848874598,
   'Recall': 0.7709923664122137,
   'F1 Score': 0.7050610820244326},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 266,
   'Precision': 0.4398496240601504,
   'Recall': 0.7597402597402597,
   'F1 Score': 0.557142857142857},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 25,
   'Precision': 0.76,
   'Recall': 0.1310344827586207,
   'F1 Score': 0.2235294117647059},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 304,
   'Precision': 0.7927631578947368,
   'Recall': 0.8763636363636363,
   'F1 Score': 0.8324697754749568},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 24,
   'Precision': 0.5,
   'Recall': 0.18461538461538463,
   'F1 Score': 0.2696629213483146},
  'OTHER': {'Actual Count': 193,
   'Predicted Count': 

In [ ]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

 17%|█▋        | 200/1151 [02:40<08:11,  1.94it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 25adcdeab2f9e2d0fbc45b2e16525a3b in your message.)
Retrying (1/5)...


 18%|█▊        | 210/1151 [03:43<17:06,  1.09s/it]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 5ee5fcca38b58be53213052889ebad84 in your message.)
Retrying (1/5)...


 26%|██▌       | 294/1151 [05:19<04:40,  3.06it/s]

...

 84%|████████▍ | 965/1151 [40:22<02:06,  1.47it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 22adb8200b4ad9de34ac2b1a1a6ba449 in your message.)
Retrying (1/5)...


 96%|█████████▌| 1107/1151 [42:59<00:17,  2.55it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 3b90c3f10ba34e51f56a775a5b57c41c in your message.)
Retrying (1/5)...


100%|██████████| 1151/1151 [44:27<00:00,  2.32s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.6159860990443093
F1 Score: 0.5888006545798635
Precision: 0.6427744638070204
Recall: 0.6159860990443093

Class-wise Counts:
Class: SAD
Actual Count: 262
Predicted Count: 313
Precision: 0.6549520766773163
Recall: 0.7824427480916031
F1 Score: 0.7130434782608696

Class: ANGRY
Actual Count: 154
Predicted Count: 256
Precision: 0.4375
Recall: 0.7272727272727273
F1 Score: 0.5463414634146341

Class: SURPRISE
Actual Count: 145
Predicted Count: 24
Precision: 0.7916666666666666
Recall: 0.1310344827586207
F1 Score: 0.22485207100591717

Class: HAPPY
Actual Count: 275
Predicted Count: 299
Precision: 0.7993311036789298
Recall: 0.8690909090909091
F1 Score: 0.832752613240418

Class: HATE
Actual Count: 65
Predicted Count: 26
Precision: 0.5
Recall: 0.2
F1 Score: 0.28571428571428575

Class: OTHER
Actual Count: 193
Predicted Count: 190
Precision: 0.45263157894736844
Recall: 0.44559585492227977
F1 Score: 0.44908616187989553

Class: FEAR
Actual Count: 57
Predicted Count: 43
Precision: 0.8139534883

{'Accuracy': 0.6159860990443093,
 'F1 Score': 0.5888006545798635,
 'Precision': 0.6427744638070204,
 'Recall': 0.6159860990443093,
 'Class-wise Counts': {'SAD': {'Actual Count': 262,
   'Predicted Count': 313,
   'Precision': 0.6549520766773163,
   'Recall': 0.7824427480916031,
   'F1 Score': 0.7130434782608696},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 256,
   'Precision': 0.4375,
   'Recall': 0.7272727272727273,
   'F1 Score': 0.5463414634146341},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 24,
   'Precision': 0.7916666666666666,
   'Recall': 0.1310344827586207,
   'F1 Score': 0.22485207100591717},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 299,
   'Precision': 0.7993311036789298,
   'Recall': 0.8690909090909091,
   'F1 Score': 0.832752613240418},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 26,
   'Precision': 0.5,
   'Recall': 0.2,
   'F1 Score': 0.28571428571428575},
  'OTHER': {'Actual Count': 193,
   'Predicted Count': 190,
   'Pre

In [ ]:
from tqdm import tqdm
from googletrans import Translator

actual = []
predicted = []

prompt = f"""
The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            translator = Translator()
            result = translator.translate(row['text'], src='fa', dest='en')

            response = get_completion(prompt.replace("SAMPLE_TWEET", result.text))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            if "JSON" not in str(e):
              print(f"An error occurred: {str(e)}")
              print(f"Retrying ({retries+1}/{max_retries})...")
            retries += 1
            time.sleep(10)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

 13%|█▎        | 153/1151 [04:37<08:26,  1.97it/s]

An error occurred: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Sun, 18 Jun 2023 09:58:01 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7d92a4279e6a4564-ATL', 'alt-svc': 'h3=":443"; ma=86400'}
Retrying (1/5)...


 15%|█▍        | 170/1151 [10:40<21:16,  1.30s/it]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 69d1550f8f14b31b91716249c7dd8e30 in your message.)
Retrying (1/5)...


 15%|█▌        | 173/1151 [11:21<1:47:33,  6.60s/it]

...

 89%|████████▉ | 1028/1151 [50:03<01:49,  1.12it/s]

An error occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 63fbaa152c2085c227857b430a4e7851 in your message.)
Retrying (1/5)...


 97%|█████████▋| 1111/1151 [53:25<00:48,  1.20s/it]

An error occurred: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Sun, 18 Jun 2023 10:46:48 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7d92eba4bcd3b0af-ATL', 'alt-svc': 'h3=":443"; ma=86400'}
Retrying (1/5)...


100%|██████████| 1151/1151 [1:00:08<00:00,  3.14s/it]


In [ ]:
calculate_metrics(actual, predicted)

Accuracy: 0.48566463944396177
F1 Score: 0.4834624468515466
Precision: 0.6096234253510274
Recall: 0.48566463944396177

Class-wise Counts:
Class: SAD
Actual Count: 262
Predicted Count: 188
Precision: 0.7287234042553191
Recall: 0.5229007633587787
F1 Score: 0.6088888888888889

Class: ANGRY
Actual Count: 154
Predicted Count: 159
Precision: 0.4088050314465409
Recall: 0.42207792207792205
F1 Score: 0.41533546325878595

Class: SURPRISE
Actual Count: 145
Predicted Count: 8
Precision: 0.625
Recall: 0.034482758620689655
F1 Score: 0.06535947712418301

Class: HAPPY
Actual Count: 275
Predicted Count: 190
Precision: 0.8
Recall: 0.5527272727272727
F1 Score: 0.6537634408602151

Class: HATE
Actual Count: 65
Predicted Count: 38
Precision: 0.631578947368421
Recall: 0.36923076923076925
F1 Score: 0.46601941747572817

Class: OTHER
Actual Count: 193
Predicted Count: 528
Precision: 0.2765151515151515
Recall: 0.7564766839378239
F1 Score: 0.40499306518724

Class: FEAR
Actual Count: 57
Predicted Count: 40
Precisio

{'Accuracy': 0.48566463944396177,
 'F1 Score': 0.4834624468515466,
 'Precision': 0.6096234253510274,
 'Recall': 0.48566463944396177,
 'Class-wise Counts': {'SAD': {'Actual Count': 262,
   'Predicted Count': 188,
   'Precision': 0.7287234042553191,
   'Recall': 0.5229007633587787,
   'F1 Score': 0.6088888888888889},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 159,
   'Precision': 0.4088050314465409,
   'Recall': 0.42207792207792205,
   'F1 Score': 0.41533546325878595},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 8,
   'Precision': 0.625,
   'Recall': 0.034482758620689655,
   'F1 Score': 0.06535947712418301},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 190,
   'Precision': 0.8,
   'Recall': 0.5527272727272727,
   'F1 Score': 0.6537634408602151},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 38,
   'Precision': 0.631578947368421,
   'Recall': 0.36923076923076925,
   'F1 Score': 0.46601941747572817},
  'OTHER': {'Actual Count': 193,
   'Predicted 